## Kernel Usage

As mentionnned in the *Basic Usage* section, the default parameters of the algorithm assume that the spatial coordinates are geolocations and that time coordinates have only one dimension. On top of that we chose some kernels by default to simplify basic usage.

When the user wants to try different kernels, it is pretty simple to do. The user can chose any class inherithing from the abstract `Kernel` (in the *Kernel Module*) and use it for spatial or temporal points. Some notable kernels that we can find in the *Kernel Module* are:

* `KernelWhiteNoise`
* `KernelDotProduct`
* `KernelSE`
* `KernelRQ`
* `KernelPeriodic`
* `KernelMatern`

Each kernel have different parameters (`KernelParameter`) that we can also individually configure. We can configure their initial values (`value`), their sampling bounds (`lower_bound`, `upper_bound`) and we can even keep those parameter constant during the whole sampling (set `is_constant` to `True`).

According to the nature of the coordinates, the user can also chose which distance type `DIST_TYPE` he wants to use when calculating the distance matrices from the `kernel_x` passed to the regressor. Let's say we have geographical coordinates, it makes sense to use `DIST_TYPE.HAVERSINE` but not so much sense to use `DIST_TYPE.LINEAR`. Our general rule of thumb would be the following:

* Use `DIST_TYPE.HAVERSINE` for geographical locations
* Use `DIST_TYPE.EUCLIDEAN` for coordinates 2D coordinates (not geographical) or coordinates with 3+ dimensions
* Use `DIST_TYPE.LINEAR` if the coordinates only have one dimension (example time points).

### Custom Kernels

To setup your kernel, you simply need to choose the kernel that interest you and initialize with the right parameters. Once initialized, you need to pass it to the right `BKTRRegressor` parameter (`spatial_kernel` or `temporal_kernel`).

The following is an example of how to setup the kernel that was presented in subsection `5.2` of the `BKTR Article`.

#### Example Spatial Kernel

The spatial kernel presented for the BIXI data in the article is a Matern 3/2 using a haversine distance. Let's see how to setup this kernel.

In [ ]:
from pyBKTR.kernels import KernelMatern
from pyBKTR.distances import DIST_TYPE

# Create a spatial matern kernel, here the smoothness factor of 3 represent a Matern 3/2
spatial_kernel = KernelMatern(smoothness_factor=3, distance_type=DIST_TYPE.HAVERSINE)

#### Example Temporal Kernel

The temporal kernel used was a little bit more complex. It was defined as a product of a periodic kernel and a SE kernel. Also, the period of the periodic kernel was deemed to be equal to 7. Let's define this kernel.

In [ ]:
from pyBKTR.kernels import KernelParameter, KernelPeriodic, KernelSE
from pyBKTR.distances import DIST_TYPE

# Since kernel use a time axis, we can always use the distance type DIST_TYPE.LINEAR
dist = DIST_TYPE.LINEAR

# Start by creating a temporal periodic kernel with a period length of 7 days
# Since the period length parameter stays constant, we need to change its definition
period_length_param = KernelParameter(7, name='period length', is_constant=True)
periodic_kernel = KernelPeriodic(period_length=period_length_param, distance_type=dist)

# Then we can create the SE kernel
se_kernel = KernelSE(distance_type=dist)

# Finally we can simply multiply the two kernels to create a new product kernel
temporal_kernel = periodic_kernel * se_kernel

Note that the multiplication of two kernels is taken in charge by the `ComposedKernel` that can be found in the `kernel` submodule of pyBKTR. Also, it is interesting to know that composed kernel can work with the addition of two kernels.

Note also that `DIST_TYPE.LINEAR` is the default `distance_type` for SE & Periodic Kernels. Thus we would have been able to further simplify those kernels' initialization (see `Example Custom Kernel Usage`).

#### Custom Kernel Usage

We can use the two kernels created previously (`spatial_kernel` and `temporal_kernel`) and use them in a `BKTRRegressor`.

In [ ]:
from pyBKTR.bktr import BKTRRegressor

bktr_regressor = BKTRRegressor(
    temporal_covariate_matrix=bixi_weather_matrix,
    spatial_covariate_matrix=bixi_station_matrix,
    y=bixi_y,
    omega=bixi_omega,
    rank_decomp=10,
    burn_in_iter=10,
    sampling_iter=5,
    spatial_kernel=spatial_kernel,
    spatial_kernel_x=spatial_kernel_x,
    temporal_kernel=temporal_kernel,
    temporal_kernel_x=temporal_kernel_x,
)
bktr_regressor.mcmc_sampling()